In [1]:
#import library 
import pandas as pd
import numpy as np
import os, cv2, shutil
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, Flatten,BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from google.colab import files
from keras.preprocessing import image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Definisikan path kaggle json
# Sesuaikan dengan path anda
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/dataset_tugas2"

In [3]:
# Ubah lokasi direktori kerja
# Sesuaikan dengan path anda
%cd /content/drive/My Drive/dataset_tugas2

/content/drive/My Drive/dataset_tugas2


In [4]:
# Cek apakah api sudah terbaca oleh sistem
!ls  

Dataset  kaggle.json  rice_leaf_diseases


In [6]:
!kaggle datasets download -d vbookshelf/rice-leaf-diseases

 87% 32.0M/36.7M [00:01<00:00, 17.5MB/s]
100% 36.7M/36.7M [00:01<00:00, 20.6MB/s]


In [7]:
# Cek isi direktori kerja dan memastikan dataset telah berhasil didownload.
!ls

kaggle.json  rice-leaf-diseases.zip


In [8]:
# Ekstrak file dataset dan hapus file zip dataset agar tidak memakan banyak tempat.
!unzip \*.zip &> /dev/null && rm *.zip

In [16]:
!ls

Dataset  kaggle.json  rice_leaf_diseases


In [ ]:
os.makedirs('/content/drive/My Drive/dataset_tugas2/Dataset')

In [6]:
!pip install split-folders

In [10]:
import splitfolders


base_dir = '/content/drive/MyDrive/dataset_tugas2/rice_leaf_diseases'
data_dir = '/content/drive/My Drive/dataset_tugas2/Dataset'

splitfolders.ratio(base_dir, output=data_dir, seed=1337, ratio=(.8, .19, .01),group_prefix=None)

Copying files: 120 files [00:29,  4.00 files/s]


In [11]:
# TULIS KODE ANDA DISINI
datagen = ImageDataGenerator(rescale=1./255)

In [12]:
input_shape = (150, 150)

train = '/content/drive/MyDrive/dataset_tugas2/Dataset/train'
train_generator = datagen.flow_from_directory(train,
                                              target_size=input_shape,
                                              class_mode='categorical',
                                              batch_size=32,
                                              color_mode='rgb',
                                              shuffle= True)

validation = '/content/drive/MyDrive/dataset_tugas2/Dataset/val'
validation_generator = datagen.flow_from_directory(validation,
                                              target_size=input_shape,
                                              class_mode='categorical',
                                              batch_size=32,
                                              color_mode='rgb',
                                              shuffle= True)

test = '/content/drive/MyDrive/dataset_tugas2/Dataset/test'
test_generator = datagen.flow_from_directory(test,
                                              target_size=input_shape,
                                              batch_size=32,
                                              color_mode='rgb',
                                              shuffle= False)

Found 96 images belonging to 3 classes.
Found 21 images belonging to 3 classes.
Found 3 images belonging to 3 classes.


In [38]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, MaxPool2D, Flatten, BatchNormalization, Dropout, GlobalMaxPooling2D, BatchNormalization

In [67]:
# Model building
#Instatiating A convnet

model = Sequential()
model.add(Conv2D(16, (3,3), input_shape=(150,150,3), activation="relu"))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(BatchNormalization())

model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(BatchNormalization())

model.add(Conv2D(64, (3,3), activation="relu"))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(BatchNormalization())

model.add(Dropout(0.2))
model.add(Flatten())


In [68]:
# Fully Connected Layer
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(3, activation="softmax"))

In [69]:
print(model.summary())

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_31 (Conv2D)          (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 74, 74, 16)       0         
 g2D)                                                            
                                                                 
 batch_normalization_17 (Bat  (None, 74, 74, 16)       64        
 chNormalization)                                                
                                                                 
 conv2d_32 (Conv2D)          (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 36, 36, 32)       0         
 g2D)                                                            
                                                     

In [70]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(lr=0.001), 
              loss='categorical_crossentropy', 
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [71]:
H = model.fit(train_generator, batch_size=2, epochs = 100, validation_data = validation_generator)

Epoch 1/100
3/3 [==============================] - 6s 2s/step - loss: 4.0689 - acc: 0.3646 - val_loss: 2.3286 - val_acc: 0.3333
Epoch 2/100
3/3 [==============================] - 4s 1s/step - loss: 1.7884 - acc: 0.7917 - val_loss: 3.3438 - val_acc: 0.3333
Epoch 3/100
3/3 [==============================] - 4s 1s/step - loss: 0.6136 - acc: 0.8125 - val_loss: 3.5030 - val_acc: 0.3333
Epoch 4/100
3/3 [==============================] - 4s 1s/step - loss: 0.1617 - acc: 0.9271 - val_loss: 3.4019 - val_acc: 0.3333
Epoch 5/100
3/3 [==============================] - 4s 1s/step - loss: 0.4187 - acc: 0.9375 - val_loss: 2.9743 - val_acc: 0.3333
Epoch 6/100
3/3 [==============================] - 4s 1s/step - loss: 0.1062 - acc: 0.9688 - val_loss: 2.4529 - val_acc: 0.3333
Epoch 7/100
3/3 [==============================] - 4s 1s/step - loss: 0.1627 - acc: 0.9375 - val_loss: 2.0165 - val_acc: 0.3333
Epoch 8/100
3/3 [==============================] - 4s 1s/step - loss: 0.0506 - acc: 0.9896 - val_loss: 1